# Тестирование RLAIF (LLM как судья)

Reinforcement Learning from AI Feedback:
- LLM оценивает качество ответов
- Автоматическое обучение на основе оценок
- Измерение улучшения системы

Требуется: OPENROUTER_API_KEY или VLLM сервер

In [ ]:
import sys
from pathlib import Path
import os

grapharchitect_path = Path.cwd().parent.parent / "src" / "GraphArchitectLib"
sys.path.insert(0, str(grapharchitect_path))

# Проверка API ключа
HAS_API_KEY = bool(os.getenv('OPENROUTER_API_KEY'))

if HAS_API_KEY:
    print("✓ OPENROUTER_API_KEY установлен - будут реальные оценки")
else:
    print("⚠ OPENROUTER_API_KEY не установлен - симуляция")
    print("  Для реальных оценок: set OPENROUTER_API_KEY=your-key")

✓ OPENROUTER_API_KEY установлен - будут реальные оценки


## 1. Создание LLM Критика

In [2]:
from grapharchitect.services.rlaif.llm_critic import LLMCritic

if HAS_API_KEY:
    critic = LLMCritic(
        backend="openrouter",
        model_name="openai/gpt-3.5-turbo",
        temperature=0.2,  # Низкая для consistency
        detailed_evaluation=True
    )
    
    print("✓ LLM Критик создан")
    print(f"  Модель: openai/gpt-3.5-turbo")
    print(f"  Температура: 0.2")
else:
    print("Симуляция LLM критика")
    critic = None

INFO:faiss.loader:Loading faiss with AVX2 support.
INFO:faiss.loader:Successfully loaded faiss with AVX2 support.
INFO:grapharchitect.tools.ApiTools.OpenRouterTool.openrouter_llm:OpenRouter инициализирован: модель openai/gpt-3.5-turbo
INFO:grapharchitect.services.rlaif.llm_critic:OpenRouter backend initialized successfully
INFO:grapharchitect.services.rlaif.llm_critic:LLM Critic initialized: backend=openrouter, model=openai/gpt-3.5-turbo, detailed=True


✓ LLM Критик создан
  Модель: openai/gpt-3.5-turbo
  Температура: 0.2


## 2. Тест оценки хорошего ответа

In [3]:
task = "Классифицировать отзыв по тональности"

good_answer = """[Классификация]
Отзыв: ПОЗИТИВНЫЙ

Обоснование:
- Фразы "отличный продукт", "очень доволен" указывают на позитив
- Нет негативных формулировок
- Общий тон восторженный

Уверенность: 95%"""

print(f"Задача: {task}")
print(f"\nОтвет:\n{good_answer}\n")

if HAS_API_KEY and critic:
    print("Оценка LLM критиком...\n")
    
    score = critic.evaluate_answer(task, good_answer)
    
    print("Результаты оценки:")
    print(f"  Общий балл: {score.overall_score:.2f}/1.0")
    print(f"  Правильность: {score.correctness:.2f}")
    print(f"  Полнота: {score.completeness:.2f}")
    print(f"  Релевантность: {score.relevance:.2f}")
    print(f"  Ясность: {score.clarity:.2f}")
    print(f"\n  Обоснование: {score.reasoning}")
else:
    print("Симуляция оценки: ~0.90 (хороший ответ)")

Задача: Классифицировать отзыв по тональности

Ответ:
[Классификация]
Отзыв: ПОЗИТИВНЫЙ

Обоснование:
- Фразы "отличный продукт", "очень доволен" указывают на позитив
- Нет негативных формулировок
- Общий тон восторженный

Уверенность: 95%

Оценка LLM критиком...



INFO:grapharchitect.tools.ApiTools.OpenRouterTool.openrouter_llm:Использовано токенов: prompt=648, completion=114, total=762
INFO:grapharchitect.services.rlaif.llm_critic:Evaluation completed: task_id=None, overall=0.900, correctness=1.000, completeness=0.900, relevance=1.000, clarity=0.900


Результаты оценки:
  Общий балл: 0.90/1.0
  Правильность: 1.00
  Полнота: 0.90
  Релевантность: 1.00
  Ясность: 0.90

  Обоснование: The answer provides a correct and accurate classification of the review's sentiment, supported by relevant evidence. It covers all essential aspects of the task with a high level of relevance and clarity.


## 3. Тест оценки плохого ответа

In [4]:
bad_answer = "Позитивный."  # Слишком короткий, нет обоснования

print(f"Задача: {task}")
print(f"\nОтвет: {bad_answer}\n")

if HAS_API_KEY and critic:
    print("Оценка LLM критиком...\n")
    
    score = critic.evaluate_answer(task, bad_answer)
    
    print("Результаты оценки:")
    print(f"  Общий балл: {score.overall_score:.2f}/1.0")
    print(f"  Правильность: {score.correctness:.2f}")
    print(f"  Полнота: {score.completeness:.2f} ⚠ (низкая)")
    print(f"  Релевантность: {score.relevance:.2f}")
    print(f"  Ясность: {score.clarity:.2f} ⚠ (низкая)")
    print(f"\n  Обоснование: {score.reasoning}")
    
    if score.suggestions:
        print(f"  Предложения: {score.suggestions}")
else:
    print("Симуляция оценки: ~0.45 (плохой ответ - неполный)")

Задача: Классифицировать отзыв по тональности

Ответ: Позитивный.

Оценка LLM критиком...



INFO:grapharchitect.tools.ApiTools.OpenRouterTool.openrouter_llm:Использовано токенов: prompt=558, completion=117, total=675
INFO:grapharchitect.services.rlaif.llm_critic:Evaluation completed: task_id=None, overall=0.900, correctness=1.000, completeness=0.800, relevance=1.000, clarity=1.000


Результаты оценки:
  Общий балл: 0.90/1.0
  Правильность: 1.00
  Полнота: 0.80 ⚠ (низкая)
  Релевантность: 1.00
  Ясность: 1.00 ⚠ (низкая)

  Обоснование: The response is correct in identifying the sentiment as positive, relevant to the task, clear, and logically sound. It is comprehensive but could benefit from mentioning the methodology or features used for sentiment analysis.
  Предложения: To improve completeness, consider mentioning the specific features or methods used for sentiment analysis to provide a more detailed and comprehensive response.


## 4. RLAIF Обучение

In [ ]:
# Создаем полный workflow с обучением
from grapharchitect.services.rlaif.rlaif_trainer import RLAIFTrainer
from grapharchitect.services.training.training_orchestrator import TrainingOrchestrator
from grapharchitect.services.execution.execution_orchestrator import ExecutionOrchestrator
from grapharchitect.services.selection.instrument_selector import InstrumentSelector
from grapharchitect.services.graph_strategy_finder import GraphStrategyFinder
from grapharchitect.entities.task_definition import TaskDefinition
from grapharchitect.entities.connectors.connector import Connector

# Инициализация
training_orch = TrainingOrchestrator(learning_rate=0.02)
selector = InstrumentSelector(temperature_constant=1.0)
finder = GraphStrategyFinder()
orchestrator = ExecutionOrchestrator(embedding, selector, finder)

if HAS_API_KEY and critic:
    rlaif_trainer = RLAIFTrainer(
        llm_critic=critic,
        training_orchestrator=training_orch
    )
    
    print("✓ RLAIF Trainer инициализирован")
else:
    print("Симуляция RLAIF")

In [ ]:
# Выполнение и оценка
task_def = TaskDefinition(
    description="Классифицировать отзыв",
    input_connector=Connector("text", "question"),
    output_connector=Connector("text", "answer"),
    input_data="Отличный продукт!"
)

# Выполнение
context = orchestrator.execute_task(task_def, tools[:1], path_limit=1, top_k=1)

print(f"Выполнено: {context.status.value}")
print(f"Результат: {context.result}")

# RLAIF оценка и обучение
if HAS_API_KEY and rlaif_trainer:
    print("\nRLAIF оценка и обучение...")
    
    result = rlaif_trainer.evaluate_and_train(
        context=context,
        task_description=task_def.description,
        result=context.result
    )
    
    print(f"\nОценка: {result.average_score:.2f}")
    print(f"Инструментов обучено: {result.tools_updated}")
    print("\nИзменения репутации:")
    for tool_name, delta in result.improvements.items():
        print(f"  {tool_name}: {delta:+.4f}")
else:
    print("\nСимуляция: Оценка 0.87, репутация +0.0174")

## Итоги

**RLAIF протестирован**:
- ✓ LLM критик оценивает по 4 критериям
- ✓ Автоматическое обучение
- ✓ Обновление репутации
- ✓ Без участия человека

**Преимущества**:
- Масштабируемость (тысячи оценок)
- Скорость (мгновенная обратная связь)
- Стоимость ($0.001-0.01 за оценку)
- Consistency (одинаковые критерии)

**Применение**:
- Автоматическая валидация
- Continuous обучение
- A/B тестирование инструментов